# Setup for PyTorch Mask-RCNN

## Installs

### Installing PyTorch

On macOS, system Python was used:
* `/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/`


In [1]:
# What Python

import sys
sys.executable

'/Library/Frameworks/Python.framework/Versions/3.6/bin/python3'

In [2]:
# # If not built for the system already, uncomment and install
# ! pip install torch==0.4.0 torchvision

In [15]:
# What PyTorch

import torch
torch.__version__

'1.0.0a0+ff608a9'

## Building the extensions

**ONLY Run A or B for each extension**

You may need to respecify your C and C++ compilers (CC, CXX).

### Option A. Non-Maximum Suppression (NMS) with CUDA support

In [ ]:
! export PATH=/Developer/NVIDIA/CUDA-9.2/bin:$PATH &&\
    export LD_LIBRARY_PATH=//Developer/NVIDIA/CUDA-9.2/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/Developer/NVIDIA/CUDA-9.2/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-9.2 &&\
    export CUDNN_LIB_DIR=//Developer/NVIDIA/CUDA-9.2/lib &&\
    export USE_CUDA=1 &&\
    export CC=clang &&\
    export CXX=clang++ &&\
    cd nms/src/cuda/ && nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    CC=clang CXX=clang++ python3 build.py

### Option B. Non-Maximum Suppression (NMS) for CPU

In [1]:
! cd nms && NO_CUDA=1 NO_NNPACK=1 MACOSX_DEPLOYMENT_TARGET=10.13 CC=clang CXX=clang++ python3 build.py

Traceback (most recent call last):
  File "build.py", line 3, in <module>
    from torch.utils.ffi import create_extension
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/ffi/__init__.py", line 1, in <module>
    raise ImportError("torch.utils.ffi is deprecated. Please use cpp extensions instead.")
ImportError: torch.utils.ffi is deprecated. Please use cpp extensions instead.


### Option A. Region-of-Interest (ROI) Alignment with CUDA support

For the next cuda compiler command, `-arch` flag refers to architectures listed at https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-compilation.

In [ ]:
! export PATH=/Developer/NVIDIA/CUDA-10.0/bin:$PATH &&\
    export LD_LIBRARY_PATH=//Developer/NVIDIA/CUDA-10.0/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/Developer/NVIDIA/CUDA-10.0/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-10.0 &&\
    export CUDNN_LIB_DIR=//Developer/NVIDIA/CUDA-10.0/lib &&\
    export USE_CUDA=1 &&\
    export CC=clang &&\
    export CXX=clang++ &&\
    cd roialign/roi_align/src/cuda/ &&\
    nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    python3 build.py

### Option B. Region-of-Interest (ROI) Alignment for CPU

**Funny note.  Built this extension under PyTorch 0.4.0 as it doesn't build under >=0.4.1**

In [17]:
! cd roialign/roi_align && NO_CUDA=1 NO_NNPACK=1 MACOSX_DEPLOYMENT_TARGET=10.9 CC=clang CXX=clang++ python3 build.py

/Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align
generating /var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmpl8k7t1fg/_crop_and_resize.c
setting the current directory to '/var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmpl8k7t1fg'
running build_ext
building '_crop_and_resize' extension
creating Users
creating Users/micheleenharris
creating Users/micheleenharris/Documents
creating Users/micheleenharris/Documents/bin
creating Users/micheleenharris/Documents/bin/github
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align/src
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch x86_6

In [ ]:
! ls

## `pycocotools` and `coco` libraries

In [10]:
%%bash

rm -fr coco
rm -fr cocoapi

export CC=clang &&\
export CXX=clang++ &&\

# git clone https://github.com/waleedka/coco.git &&\
git clone https://github.com/cocodataset/cocoapi.git &&\
cd cocoapi &&\
cd PythonAPI &&\
python3 setup.py build_ext --inplace &&\
cd ../../ &&\
ln -s coco/PythonAPI/pycocotools/ pycocotools

running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.macosx-10.9-x86_64-3.6
creating build/temp.macosx-10.9-x86_64-3.6/pycocotools
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch x86_64 -g -I/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include -I../common -I/Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c ../common/maskApi.c -o build/temp.macosx-10.9-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch x86_64 -g -I/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include -I../common -I/Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c pycocotools/_ma

Cloning into 'cocoapi'...
1 warning generated.
In file included from pycocotools/_mask.c:567:
In file included from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1821:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: "Using deprecated NumPy API, disable it by "          "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it by " \
 ^
2 warnings generated.
ln: pycocotools: File exists


In [20]:
! pip install -r requirements.txt

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Note, here, for macOS, using Xcode 9.4.1 command line tools

In [19]:
# # MacOS, see https://pytorch.org/ for other distributions
! pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
    100% |████████████████████████████████| 24.2MB 955kB/s ta 0:00:011
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Clean up
! rm -fr pycocotools &&\
    rm -fr coco

More relevant resolved GitHub issues:
* https://github.com/uber/horovod/issues/274#issuecomment-390813445